<a href="https://colab.research.google.com/github/ronakdm/input-marginalization/blob/main/lstm_sst2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/ronakdm/input-marginalization.git

In [2]:
%%bash
cd input-marginalization
git pull
cd ..

Already up to date.


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
save_dir = "/content/gdrive/My Drive/input-marginalization"

Mounted at /content/gdrive


In [6]:
import pickle
import numpy as np
import time
import datetime
import random
import torch
import torch.nn as nn

In [7]:
import sys
sys.path.append("input-marginalization")

from utils import generate_dataloaders, train, test
from models import LSTM

In [8]:
%%capture
try:
    from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup, BertTokenizer
except ModuleNotFoundError:
    !pip install transformers
    from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup, BertTokenizer
    

In [9]:
LEARNING_RATE = 1e-4
ADAMW_TOLERANCE = 1e-8
BATCH_SIZE = 10
EPOCHS = 30
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
VOCAB_SIZE = tokenizer.vocab_size
LABEL_SIZE =2
HIDDEN_DIM = 200
EMBEDDING_DIM = 100
N_RNN_LAYERS = 2

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on '%s'." % device)


Running on 'cuda'.


In [10]:
train_dataloader, validation_dataloader, test_dataloader = generate_dataloaders(BATCH_SIZE)

6,919 training samples.
  876 validation samples.
1,822 test samples.


In [11]:
model = LSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, LABEL_SIZE, N_RNN_LAYERS).to(device)
save_filename = "lstm_sst2"
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps = ADAMW_TOLERANCE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = EPOCHS * BATCH_SIZE * len(train_dataloader))

In [12]:
try:
    train(model, EPOCHS, train_dataloader, validation_dataloader, optimizer, scheduler, save_dir, save_filename, device)
except KeyboardInterrupt:
    print("Graceful Exit")


======== Epoch 1 / 30 ========
Training...
  Batch    40  of    692.    Elapsed: 0:00:02.
  Batch    80  of    692.    Elapsed: 0:00:04.
  Batch   120  of    692.    Elapsed: 0:00:05.
  Batch   160  of    692.    Elapsed: 0:00:07.
  Batch   200  of    692.    Elapsed: 0:00:09.
  Batch   240  of    692.    Elapsed: 0:00:10.
  Batch   280  of    692.    Elapsed: 0:00:12.
  Batch   320  of    692.    Elapsed: 0:00:14.
  Batch   360  of    692.    Elapsed: 0:00:15.
  Batch   400  of    692.    Elapsed: 0:00:17.
  Batch   440  of    692.    Elapsed: 0:00:19.
  Batch   480  of    692.    Elapsed: 0:00:20.
  Batch   520  of    692.    Elapsed: 0:00:22.
  Batch   560  of    692.    Elapsed: 0:00:24.
  Batch   600  of    692.    Elapsed: 0:00:25.
  Batch   640  of    692.    Elapsed: 0:00:27.
  Batch   680  of    692.    Elapsed: 0:00:29.

  Average training loss: 0.69
  Training epcoh took: 0:00:29

Running Validation...
  Accuracy: 0.55
  Validation Loss: 0.68
  Validation took: 0:00:01

===

In [13]:
test(model, test_dataloader, device, save_dir, save_filename)


Testing...
  Accuracy: 0.77
  Test Loss: 0.76
  Test took: 0:00:02
